## DIA 098: Programar Envío Automático de Reportes PDF Semanales con schedule y smtplib

Hoy automatizaremos el envío del reporte PDF semanal al correo del usuario sin intervención manual, usando la librería schedule.

✅ ¿Qué aprenderás hoy?

⏱️ Programar tareas automáticas con schedule.

🧾 Generar PDFs periódicamente.

📧 Enviar correos con smtplib.

🧩 1. Instalar schedule si no lo tienes
bash
Copiar
Editar
pip install schedule
🧩 2. Crear Script Python: tarea_semanal.py
Este archivo no va en Flask, es un script externo que puedes ejecutar con cron o mantener corriendo.

python
Copiar
Editar
import os
import schedule
import time
import smtplib
import io
from datetime import datetime, timedelta
from email.message import EmailMessage
from dotenv import load_dotenv
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from app import db, app, Registro, User  # importa tu app Flask

load_dotenv()

# ---------------------
# Función de envío
# ---------------------
def enviar_reporte_semanal():
    with app.app_context():
        hoy = datetime.today()
        hace_7_dias = hoy - timedelta(days=7)

        usuarios = User.query.all()

        for user in usuarios:
            registros = Registro.query.filter(
                Registro.user_id == user.id,
                Registro.fecha >= hace_7_dias,
                Registro.fecha <= hoy
            ).order_by(Registro.fecha.desc()).all()

            if not registros:
                print(f"No hay datos para {user.username}")
                continue

            # Generar PDF en memoria
            buffer = io.BytesIO()
            p = canvas.Canvas(buffer, pagesize=letter)
            width, height = letter

            p.setFont("Helvetica-Bold", 14)
            p.drawString(50, height - 50, f"Reporte Semanal de {user.username}")
            p.setFont("Helvetica", 12)
            p.drawString(50, height - 70, f"Desde {hace_7_dias.date()} hasta {hoy.date()}")

            y = height - 100
            p.setFont("Helvetica", 10)
            p.drawString(50, y, "Fecha        | F1 | F2 | F3 | F4 | Predicción")
            y -= 15
            p.line(50, y, width - 50, y)
            y -= 15

            for r in registros:
                linea = f"{r.fecha.strftime('%Y-%m-%d %H:%M')}  | {r.f1}  | {r.f2}  | {r.f3}  | {r.f4}  | {r.prediccion}"
                p.drawString(50, y, linea)
                y -= 15
                if y < 50:
                    p.showPage()
                    y = height - 50

            p.save()
            buffer.seek(0)

            # Enviar correo
            msg = EmailMessage()
            msg['Subject'] = f'Reporte Semanal de Predicciones ({user.username})'
            msg['From'] = os.getenv('EMAIL_SENDER')
            msg['To'] = user.email
            msg.set_content(f'Hola {user.username},\n\nAdjunto tu reporte semanal de predicciones.\n\nSaludos,\nTu App de Predicción')

            msg.add_attachment(buffer.read(), maintype='application', subtype='pdf',
                               filename=f'reporte_semanal_{user.username}.pdf')

            try:
                with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
                    smtp.login(os.getenv('EMAIL_SENDER'), os.getenv('EMAIL_PASSWORD'))
                    smtp.send_message(msg)
                print(f'✅ Enviado a {user.email}')
            except Exception as e:
                print(f'❌ Error al enviar a {user.email}: {e}')

# ---------------------
# Programar cada semana
# ---------------------
schedule.every().monday.at("08:00").do(enviar_reporte_semanal)

print("🕒 Esperando a que llegue el lunes 8am para enviar los reportes...")

while True:
    schedule.run_pending()
    time.sleep(60)
